# Analyzing Iowa Liquor Sales

## Data Set 

The state of Iowa's Alcoholic Beverages Division [dataset](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy  ) consists of spirit purchase information from January 1, 2012 to current. 

> The Division can provide this level of information because Iowa is one of 17 states that directly controls the sale and distribution of alcoholic beverages https://abd.iowa.gov/division. 

It should be noted that the data set is limited to Iowa Class “E” liquor license which is for:  

> Grocery stores, liquor stores, convenience stores, etc. Allows commercial establishments to sell liquor for off-premises consumption in original unopened containers. No sales by the drink.

Also, while this data set is not direct consumer sales, we can use this information as a proxy indicator of sales because most stores should be only buying spirits that sell well in their stores.  

### Adding the Data Set to MongoDB

In [14]:
from pymongo import MongoClient

client = MongoClient()
db = client.test
liquor = db.iowaliquor
liquor.delete_many({})

In [15]:
import numpy as np, pandas as pd, datetime, urllib

In [21]:
from sodapy import Socrata
client = Socrata("data.iowa.gov", "ibhJMpvP9MQ1Rl0Ox5wlSMpgn")
results = client.get("spsw-4jax", limit = 5500)
added_results = liquor.insert_many(results)

In [22]:
list = []
for liquors in liquor.find():
    list.append(liquors)

In [23]:
pd.DataFrame(list)

,_id,address,bottle_volume_ml,category,category_name,city,county,county_number,date,im_desc,...,state_bottle_cost,state_bottle_retail,store,store_location,store_location_address,store_location_city,store_location_zip,vendor_name,vendor_no,zipcode
0,571ebe319c9e902fb06b5d5e,812 S 1ST AVE,1750,NaN,NaN,IOWA CITY,Johnson,52,2015-11-04T00:00:00.000,Knob Creek w/ Crystal Decanter,...,35.55,53.34,2513,NaN,812 S 1ST AVE,IOWA CITY,52240,Jim Beam Brands,65,52240
1,571ebe319c9e902fb06b5d5f,"522 MULBERRY, SUITE A",750,1701100,DECANTERS & SPECIALTY PACKAGES,MUSCATINE,Muscatine,70,2015-11-30T00:00:00.000,Laphroaig w/ Whiskey Stones,...,19.58,29.37,2662,NaN,"522 MULBERRY, SUITE A",MUSCATINE,52761,Jim Beam Brands,65,52761
2,571ebe319c9e902fb06b5d60,HIGHWAY 65 SOUTH,1500,1701100,DECANTERS & SPECIALTY PACKAGES,IOWA FALLS,Hardin,42,2015-11-09T00:00:00.000,Forbidden Secret Coffee Pack,...,11.62,17.43,2539,NaN,HIGHWAY 65 SOUTH,IOWA FALLS,50126,Duggan's Distillers Products Corp,962,50126
3,571ebe319c9e902fb06b5d61,2126 KIMBALL AVE,150,NaN,NaN,WATERLOO,Black Hawk,07,2015-11-16T00:00:00.000,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,...,6.4,9.6,2643,NaN,2126 KIMBALL AVE,WATERLOO,50701,Disaronno International LLC,130,50701
4,571ebe319c9e902fb06b5d62,520 SO FREMONT,750,NaN,NaN,SHENANDOAH,Page,73,2015-11-21T00:00:00.000,Templeton Rye w/Flask,...,18.09,27.14,2607,NaN,520 SO FREMONT,SHENANDOAH,51601,Wilson Daniels Ltd.,255,51601
5,571ebe319c9e902fb06b5d63,HIGHWAY 65 SOUTH,750,NaN,NaN,IOWA FALLS,Hardin,42,2015-11-20T00:00:00.000,Templeton Rye w/Flask,...,18.09,27.14,2539,NaN,HIGHWAY 65 SOUTH,IOWA FALLS,50126,Wilson Daniels Ltd.,255,50126
6,571ebe319c9e902fb06b5d64,2608 E. CENTRAL AVENUE,750,NaN,NaN,ESTHERVILLE,Emmet,32,2015-11-21T00:00:00.000,Templeton Rye w/Flask,...,18.09,27.14,2556,NaN,2608 E. CENTRAL AVENUE,ESTHERVILLE,51334,Wilson Daniels Ltd.,255,51334
7,571ebe319c9e902fb06b5d65,516 HWY 141,1500,1701100,DECANTERS & SPECIALTY PACKAGES,COON RAPIDS,Carroll,14,2015-11-05T00:00:00.000,Forbidden Secret Coffee Pack,...,11.62,17.43,4881,NaN,516 HWY 141,COON RAPIDS,50058,Duggan's Distillers Products Corp,962,50058
8,571ebe319c9e902fb06b5d66,4815 MAPLE DRIVE,1750,NaN,NaN,PLEASANT HILL,Polk,77,2015-11-04T00:00:00.000,Knob Creek w/ Crystal Decanter,...,35.55,53.34,2536,NaN,4815 MAPLE DRIVE,PLEASANT HILL,50317,Jim Beam Brands,65,50317
9,571ebe319c9e902fb06b5d67,2126 KIMBALL AVE,1500,1701100,DECANTERS & SPECIALTY PACKAGES,WATERLOO,Black Hawk,07,2015-11-09T00:00:00.000,Forbidden Secret Coffee Pack,...,11.62,17.43,2643,NaN,2126 KIMBALL AVE,WATERLOO,50701,Duggan's Distillers Products Corp,962,50701


In [24]:
for sale in liquor.find({"city" : "IOWA CITY"}):
 print(sale)

{u'state_bottle_cost': u'35.55', u'vendor_no': u'65', u'county': u'Johnson', u'pack': u'3', u'city': u'IOWA CITY', u'zipcode': u'52240', u'county_number': u'52', u'im_desc': u'Knob Creek w/ Crystal Decanter', u'store': u'2513', u'bottle_volume_ml': u'1750', u'sale_dollars': u'160.02', u'sale_liters': u'5.25', u'store_location_zip': u'52240', u'store_location_address': u'812 S 1ST AVE', u'store_location_city': u'IOWA CITY', u'address': u'812  S 1ST AVE', u'date': u'2015-11-04T00:00:00.000', u'sale_bottles': u'3', u'vendor_name': u'Jim Beam Brands', u'name': u'Hy-Vee Food Store #2 / Iowa City', u'state_bottle_retail': u'53.34', u'itemno': u'237', u'_id': ObjectId('571ebe319c9e902fb06b5d5e'), u'sale_gallons': u'1.38690327488', u'invoice_line_no': u'S28867700001'}
{u'state_bottle_cost': u'35.55', u'vendor_no': u'65', u'county': u'Johnson', u'pack': u'3', u'city': u'IOWA CITY', u'zipcode': u'52240', u'county_number': u'52', u'im_desc': u'Knob Creek w/ Crystal Decanter', u'store': u'2513', u